# Library 설치

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import os, json, time, html, re, random, csv
from datetime import datetime
from urllib.parse import urlparse, parse_qs, unquote
from typing import Dict, List, Optional

# 1. 드라이버 초기화

In [2]:
# --- 1. 드라이버 초기화 (로컬 Windows, 내 프로필 미사용) ---
# 소수점 아래 2자리까지 표시하도록 설정
pd.options.display.float_format = '{:.2f}'.format

def create_driver(headless=False, remote_debug_port=9222):
    opts = Options()
    # ✅ 안정화 필수 옵션 (Windows에서도 유효)
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-gpu")
    opts.add_argument(f"--remote-debugging-port={remote_debug_port}")
    opts.add_argument("--lang=ko_KR")
    opts.add_argument("window-size=1920x1080")
    # UA가 필요하면 아래 주석 해제
    # opts.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36")

    # ❌ 내 크롬 프로필 사용 금지 (충돌/잠금 방지)
    # opts.add_argument(f"user-data-dir=...")  # 사용하지 않음

    if headless:
        # 최신 크롬은 headless=new 권장
        opts.add_argument("--headless=new")

    # 자동화 흔적 최소화
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=opts)
    try:
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    except Exception:
        pass
    return driver

try:
    driver = create_driver(headless=False, remote_debug_port=9222)
except Exception as e:
    print(f"드라이버 초기화 중 오류 발생: {e}")
    raise


# 2. 검색 페이지 이동

In [3]:
# --- 2. 검색 페이지 이동 ---
search_url = "https://search.shopping.naver.com/" # 예시 URL
driver.get(search_url)

# 함수 세팅

In [4]:
def smooth_scroll_by(driver, scroll_by_y, duration):
    """
    주어진 픽셀(scroll_by_y)만큼 주어진 시간(duration) 동안 부드럽게 스크롤합니다.
    """
    start_time = time.time()
    current_y = driver.execute_script("return window.scrollY")
    target_y = current_y + scroll_by_y

    while time.time() < start_time + duration:
        # 경과 시간을 기반으로 진행률 계산 (0.0 to 1.0)
        progress = (time.time() - start_time) / duration
        
        # 진행률이 1을 넘지 않도록 보정
        progress = min(progress, 1.0)

        # Ease-in-out 효과: 시작과 끝을 부드럽게 만듭니다.
        # sin 함수를 이용해 S자 곡선을 그려 가속/감속 효과를 냅니다.
        eased_progress = 0.5 * (1 - math.cos(progress * math.pi))
        
        # 현재 스크롤 위치 계산
        scroll_to = current_y + (scroll_by_y * eased_progress)
        driver.execute_script(f"window.scrollTo(0, {scroll_to})")
        
        # 브라우저가 렌더링할 시간을 주기 위한 짧은 대기
        time.sleep(0.01)

    # 오차 보정을 위해 스크롤이 끝나면 목표 지점으로 정확히 이동
    driver.execute_script(f"window.scrollTo(0, {target_y})")

def advanced_smart_scroll(driver):
    """
    [최종 버전] 새로운 콘텐츠 로드를 보장하며, 사람처럼 부드러운 속도로 스크롤합니다.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # 스크롤할 거리와 시간을 랜덤하게 설정
        scroll_distance = random.randint(800, 1200)
        scroll_duration = random.uniform(2.0, 3.5) # 2.0초 ~ 3.5초 사이

        print(f"{scroll_distance}px를 {scroll_duration:.1f}초 동안 부드럽게 스크롤합니다...")
        smooth_scroll_by(driver, scroll_distance, scroll_duration)
        
        # 새 콘텐츠가 로드될 시간을 충분히 줍니다.
        time.sleep(random.uniform(3.0, 5.0))

        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            print("페이지 높이가 더 이상 늘어나지 않아 스크롤을 종료합니다.")
            break
            
        last_height = new_height


# 3. 상품 목록 수집

In [16]:
# --- 3. 상품 목록 수집 (무한 스크롤 고려 + 셀렉터 최신화 + 할인율 소수) ---

wait = WebDriverWait(driver, 20)

# ===== 체크포인트 설정 =====
CHECKPOINT_DIR = "./checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_CSV  = os.path.join(CHECKPOINT_DIR, "df_list_checkpoint.csv")   # 누적 DataFrame
CHECKPOINT_META = os.path.join(CHECKPOINT_DIR, "progress.json")            # 진행 메타(라운드, 저장시각 등)

SAVE_EVERY_SEC = 30          # 최소 n초마다 한 번 저장
SAVE_EVERY_N   = 50          # 또는 n개 이상 새로 수집되면 저장
last_save_time = time.time()
last_saved_rows_count = 0    # 직전 저장 시점의 누적 rows 개수

def _safe_write(path: str, write_fn):
    """임시 파일로 쓴 뒤 원자적으로 교체 → 부분 쓰기 방지"""
    tmp = path + ".tmp"
    write_fn(tmp)
    os.replace(tmp, path)

def save_checkpoint(rows: list, rounds: int, extra: dict = None):
    """rows를 CSV로, 메타를 JSON으로 저장"""
    global last_save_time, last_saved_rows_count
    df_ckpt = pd.DataFrame(rows)
    def _write_csv(p): df_ckpt.to_csv(p, index=False, encoding="utf-8-sig")
    _safe_write(CHECKPOINT_CSV, _write_csv)

    meta = {
        "rounds": rounds,
        "saved_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "rows_count": len(rows),
    }
    if extra:
        meta.update(extra)
    def _write_json(p): 
        with open(p, "w", encoding="utf-8") as f: json.dump(meta, f, ensure_ascii=False, indent=2)
    _safe_write(CHECKPOINT_META, _write_json)

    last_save_time = time.time()
    last_saved_rows_count = len(rows)
    print(f"💾 체크포인트 저장: {len(rows)}행 @ round {rounds}")

def maybe_autosave(rows: list, rounds: int):
    """주기/증분 기준 만족 시 자동저장"""
    now = time.time()
    if (now - last_save_time) >= SAVE_EVERY_SEC or (len(rows) - last_saved_rows_count) >= SAVE_EVERY_N:
        save_checkpoint(rows, rounds)

def resume_from_checkpoint():
    """기존 체크포인트 있으면 병합 재개: (processed, rows, rounds_start) 반환"""
    if not os.path.exists(CHECKPOINT_CSV):
        return set(), [], 0

    print("↩️  기존 체크포인트를 발견했습니다. 재개합니다.")
    df_prev = pd.read_csv(CHECKPOINT_CSV)
    # 열 표준화(없을 수 있는 열을 미리 만들어 둠)
    for col in ["상품링크", "제목", "원래가격", "현재가격", "할인율", "스토어"]:
        if col not in df_prev.columns:
            df_prev[col] = None

    # 키 생성: 링크 우선, 없으면 제목
    def _key(row):
        link = row.get("상품링크") if isinstance(row, dict) else row["상품링크"]
        title = row.get("제목") if isinstance(row, dict) else row["제목"]
        return link if (isinstance(link, str) and link.strip()) else (title if isinstance(title, str) else None)

    # 중복 제거(키 기준) → rows 복원
    seen = set()
    rows = []
    for rec in df_prev.to_dict("records"):
        k = _key(rec)
        if k and k not in seen:
            seen.add(k)
            rows.append(rec)

    # rounds 복원(있으면), 없으면 0
    rounds_start = 0
    try:
        if os.path.exists(CHECKPOINT_META):
            with open(CHECKPOINT_META, "r", encoding="utf-8") as f:
                meta = json.load(f)
                rounds_start = int(meta.get("rounds", 0))
    except:
        pass

    print(f"✅ 재개 준비 완료: {len(rows)}행 불러옴, rounds 시작 = {rounds_start}")
    return seen, rows, rounds_start


# 결과 영역 등장 대기
wait.until(EC.presence_of_element_located(
    (By.CSS_SELECTOR, 'ul[class^="compositeCardList_product_list__"]')
))

# ===== 유틸들 =====
card_selector = 'ul[class^="compositeCardList_product_list__"] > li[class^="compositeCardContainer_composite_card_container__"]'
title_sel    = 'strong[class^="productCardTitle_product_card_title__"]'
store_sel    = 'a[class^="productCardMallLink_mall_link__"] span[class^="productCardMallLink_mall_name__"]'
price_sel    = 'span[class^="priceTag_price__"]'
orig_sel     = 'span[class^="priceTag_original_price__"]'
disc_sel     = 'span[class^="priceTag_discount_ratio__"]'

def extract_smartstore_link_from_li(li):
    """
    네이버 플러스스토어 목록의 각 상품 li에서 상세페이지 링크(href) 추출.
    규칙: div.basicProductCard_basic_product_card__TdrHT 아래의
          a.basicProductCard_link__urzND[href] 가 상세 링크.
    """
    # 1) 카드 컨테이너 찾기
    container = None
    try:
        container = li.find_element(
            By.CSS_SELECTOR,
            'div.basicProductCard_basic_product_card__TdrHT'
        )
    except:
        # 클래스가 바뀐 경우 대비해 li 자체를 fallback으로 사용
        container = li

    # 2) 대표 링크 우선 추출
    try:
        a = container.find_element(
            By.CSS_SELECTOR, 'a.basicProductCard_link__urzND[href]'
        )
        href = a.get_attribute('href')
        if href:
            return href
    except:
        pass

    # 3) 보강: data-shp-contents-dtl 안의 click_url 또는 임의의 a[href]
    candidates = []
    try:
        candidates = container.find_elements(By.CSS_SELECTOR, 'a[href]')
    except:
        candidates = []

    for a in candidates:
        # data-shp-contents-dtl → click_url 시도
        try:
            meta = a.get_attribute("data-shp-contents-dtl")
            if meta:
                meta_json = json.loads(html.unescape(meta))
                for entry in meta_json:
                    if entry.get("key") == "click_url" and entry.get("value"):
                        return entry.get("value")
        except:
            pass
        # 일반 href
        try:
            href = a.get_attribute("href")
            if href:
                return href
        except:
            pass

    return None


def to_int(text):
    if not text:
        return None
    digits = re.sub(r"[^\d]", "", text)
    return int(digits) if digits else None

def parse_discount_fraction(text):
    # '34%', '34 % 할인' → 0.34, 없으면 None
    if not text:
        return None
    m = re.search(r"(\d+)", text)
    return (int(m.group(1)) / 100.0) if m else None

def extract_click_url_from(elem):
    # mall_link data JSON → href → 상단 링크 순
    anchor_selectors = [
        'a[class^="productCardMallLink_mall_link__"]',
        'a[class^="basicProductCard_link__"]',
    ]
    for sel in anchor_selectors:
        try:
            a = elem.find_element(By.CSS_SELECTOR, sel)
        except:
            continue
        try:
            meta = a.get_attribute("data-shp-contents-dtl")
            if meta:
                meta_json = json.loads(html.unescape(meta))
                for entry in meta_json:
                    if entry.get("key") == "click_url":
                        return entry.get("value")
        except:
            pass
        try:
            href = a.get_attribute("href")
            if href:
                return href
        except:
            pass
    return None

def collect_visible_cards(processed_keys, rows_out):
    """현재 화면 근방에 존재하는 카드만 수집. 이미 본 항목은 건너뜀."""
    cards = driver.find_elements(By.CSS_SELECTOR, card_selector)
    new_added = 0
    for li in cards:
        # 제목
        try:
            title = li.find_element(By.CSS_SELECTOR, title_sel).text.strip()
        except:
            continue

        # 링크 (키)
        link = extract_smartstore_link_from_li(li)
        key  = link or title  # 링크가 없으면 제목으로 임시 키

        if key in processed_keys:
            continue

        # 스토어
        try:
            store = li.find_element(By.CSS_SELECTOR, store_sel).text.strip()
        except:
            store = None

        # 가격들
        try:
            current_text = li.find_element(By.CSS_SELECTOR, price_sel).text
        except:
            current_text = None
        try:
            original_text = li.find_element(By.CSS_SELECTOR, orig_sel).text
        except:
            original_text = None
        try:
            discount_text = li.find_element(By.CSS_SELECTOR, disc_sel).text
        except:
            discount_text = None

        current_price  = to_int(current_text)
        original_price = to_int(original_text)
        discount_frac  = parse_discount_fraction(discount_text)  # 0.34 형태

        rows_out.append({
            "스토어": store,
            "제목": title,
            "원래가격": original_price,
            "현재가격": current_price,
            "할인율": discount_frac,
            "상품링크": link,
        })
        processed_keys.add(key)
        new_added += 1
    return new_added, len(cards)

# ===== 메인 루프: 스크롤 ↔ 수집 반복 =====
# 재개(Resume) 시도
processed, rows, rounds = resume_from_checkpoint()

stable_needed = random.randint(3, 5)  # 연속 정지 임계
stable_count  = 0

prev_height = driver.execute_script("return document.body.scrollHeight")
prev_total  = len(rows)  # 누적 수집 개수(재개 시 반영)
prev_domcnt = 0

start_time  = time.time()
MAX_ROUNDS  = 400
MAX_SECONDS = 1800 # 최대 30분

try:
    while True:
        # 1) 현재 보이는 카드 수집
        added, domcnt = collect_visible_cards(processed, rows)

        # 2) 스크롤 (큰 폭)
        scroll_distance = random.randint(3000, 4000)
        scroll_duration = random.uniform(2.0, 3.5)
        print(f"[round {rounds+1}] 수집 + 스크롤 {scroll_distance}px / {scroll_duration:.1f}s | 새 {added}개 | 누적 {len(rows)}개")
        smooth_scroll_by(driver, scroll_distance, scroll_duration)

        # 3) 로딩 대기
        time.sleep(random.uniform(3.0, 5.0))

        # 3.5) 자동 저장
        maybe_autosave(rows, rounds)

        # 4) 증가 여부 측정
        new_height = driver.execute_script("return document.body.scrollHeight")
        grew_height = new_height > prev_height
        grew_total  = len(rows) > prev_total
        grew_dom    = domcnt > prev_domcnt

        if not grew_height and not grew_total and not grew_dom:
            stable_count += 1
            print(f"  ↳ 증가 없음 → 연속 정지 {stable_count}/{stable_needed}")
            if stable_count >= stable_needed:
                print(f"끝까지 로드됨: 총 {len(rows)}개 수집, 라운드 {rounds+1}회")
                break
        else:
            stable_count = 0
            prev_height  = max(prev_height, new_height)
            prev_total   = len(rows)
            prev_domcnt  = domcnt

        rounds += 1
        if rounds >= MAX_ROUNDS:
            print("안전장치: 최대 라운드 도달 → 종료")
            break
        if time.time() - start_time > MAX_SECONDS:
            print("안전장치: 최대 시간 도달 → 종료")
            break

    # 마지막으로 화면에 남아있는 카드도 한 번 더 수집
    added, _ = collect_visible_cards(processed, rows)
    if added:
        print(f"마지막 추가 수집 {added}개 반영 (최종 {len(rows)}개)")

finally:
    # ✅ 종료 직전 무조건 최종 저장 (예외/중단 포함)
    save_checkpoint(rows, rounds, extra={"final": True})


df_list = pd.DataFrame(rows)
# display(df_list.head(20))
print(f"최종 수집 상품 수: {len(df_list)}")

↩️  기존 체크포인트를 발견했습니다. 재개합니다.
✅ 재개 준비 완료: 14592행 불러옴, rounds 시작 = 356
[round 357] 수집 + 스크롤 3194px / 2.9s | 새 0개 | 누적 14592개
💾 체크포인트 저장: 14592행 @ round 356
[round 358] 수집 + 스크롤 3931px / 2.0s | 새 50개 | 누적 14642개
💾 체크포인트 저장: 14642행 @ round 357
[round 359] 수집 + 스크롤 3862px / 2.8s | 새 50개 | 누적 14692개
💾 체크포인트 저장: 14692행 @ round 358
[round 360] 수집 + 스크롤 3290px / 2.1s | 새 50개 | 누적 14742개
💾 체크포인트 저장: 14742행 @ round 359
[round 361] 수집 + 스크롤 3630px / 3.4s | 새 50개 | 누적 14792개
💾 체크포인트 저장: 14792행 @ round 360
[round 362] 수집 + 스크롤 3015px / 3.4s | 새 50개 | 누적 14842개
💾 체크포인트 저장: 14842행 @ round 361
[round 363] 수집 + 스크롤 3363px / 2.9s | 새 50개 | 누적 14892개
💾 체크포인트 저장: 14892행 @ round 362
[round 364] 수집 + 스크롤 3811px / 2.1s | 새 50개 | 누적 14942개
💾 체크포인트 저장: 14942행 @ round 363
[round 365] 수집 + 스크롤 3187px / 2.4s | 새 50개 | 누적 14992개
💾 체크포인트 저장: 14992행 @ round 364
[round 366] 수집 + 스크롤 3451px / 3.2s | 새 0개 | 누적 14992개
[round 367] 수집 + 스크롤 3469px / 3.3s | 새 50개 | 누적 15042개
💾 체크포인트 저장: 15042행 @ round 366
[round 368] 수집 + 스

In [17]:
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16542 entries, 0 to 16541
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품링크    16542 non-null  object 
 1   제목      16542 non-null  object 
 2   원래가격    9048 non-null   float64
 3   현재가격    16542 non-null  int64  
 4   할인율     8499 non-null   float64
 5   스토어     16542 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 775.5+ KB


In [18]:
df_list.head()

,상품링크,제목,원래가격,현재가격,할인율,스토어
0,https://ader.naver.com/v1/iJF2KtiXRPKLoHoNfO7a...,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 1개",NaN,32000,NaN,바이브랩
1,https://ader.naver.com/v1/KqGKUbXY1m6Dx3FGjvJc...,"바이브랩 4주 솔루션 초록 탈모 샴푸 우디플로럴머스크, 500ml, 2개",NaN,62000,NaN,바이브랩
2,https://ader.naver.com/v1/tvF4JdnxrTV-LWj7jD8I...,릴리이브 지성탈모 볼륨샴푸 추천 500g대용량,32000.00,19800,0.38,릴리이브
3,https://ader.naver.com/v1/mcdBqQ4YfeEcbkrOAFbd...,(1세트) 다모애 테라피 골드 샴푸 & 헤어토닉 탈모증상완화도움,54000.00,45000,0.16,공식스토어 다모애탈모스토리
4,https://smartstore.naver.com/main/products/115...,려 루트젠 두피 에센스 대용량 145ml 두피 영양제 여성 남성 탈모 앰플,36000.00,23900,0.33,아모레퍼시픽몰 헤어바디


In [19]:
df_list.tail()

,상품링크,제목,원래가격,현재가격,할인율,스토어
16537,https://smartstore.naver.com/main/products/114...,뉴 프리미엄 TS 샴푸 500g,16000.00,15980,NaN,쇼핑끝
16538,https://smartstore.naver.com/main/products/817...,해외비비스칼 여성 모발 관리 헤어그로우 180정 viviscal,NaN,141700,NaN,리플레이스몰
16539,https://smartstore.naver.com/main/products/114...,보타믹스 포레스트 그로우 인텐시브 샴푸 500ml,19000.00,18980,NaN,쇼핑끝
16540,https://smartstore.naver.com/main/products/508...,오라라 비타민 헤어 트리트먼트,NaN,38000,NaN,스타일고어스
16541,https://smartstore.naver.com/main/products/105...,[트레이더스] TS 스칼프 샴푸 500G X 2입,NaN,27480,NaN,구매대행 마트


In [20]:
df_list.shape

(16542, 6)

In [21]:
# 중복 제거
df_list = df_list.drop_duplicates(subset=["상품링크", "제목"])
df_list.shape

(16542, 6)

In [15]:
# -- 전처리 --
# 제목 앞에 '해외' 붙은 것 -> [해외]로 변경
df_list['제목'] = df_list['제목'].str.replace(r'^해외', '[해외] ', regex=True)
df_list.tail()

,상품링크,제목,원래가격,현재가격,할인율,스토어
14587,https://smartstore.naver.com/main/products/766...,왕십리 영양 탈모관리 24회(6개월),NaN,1584000,NaN,지혜스트리
14588,https://smartstore.naver.com/main/products/988...,티에스 탈모 쿨 샴푸 TS 쿨링 500g x 4개,NaN,82500,NaN,히트릿
14589,https://smartstore.naver.com/main/products/111...,포미포미비오틴 맥주효모 탈모방지샴푸,NaN,37100,NaN,에스끌리에
14590,https://smartstore.naver.com/main/products/996...,탈모비누 두피관리 산야초 천연 허브 비누,30000.00,15000,0.50,뷰티-푸드
14591,https://smartstore.naver.com/main/products/899...,콜라겐 건성 두피 탈모 완화 샴푸 약산성,NaN,30000,NaN,이든토리


# 4. 상세 페이지 스펙 수집

In [5]:
# === [ADD] 체크포인트 설정/유틸 ===

CHECKPOINT_DIR = "checkpoints_step4"
RESULTS_CSV = os.path.join(CHECKPOINT_DIR, "step4_specs_results.csv")
PROCESSED_JSON = os.path.join(CHECKPOINT_DIR, "step4_processed_urls.json")
LAST_LOG = os.path.join(CHECKPOINT_DIR, "step4_last_checkpoint.txt")

def _ensure_dirs():
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)

def _append_row_csv(row: dict, path: str = RESULTS_CSV):
    _ensure_dirs()
    write_header = not os.path.exists(path)
    with open(path, "a", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=list(row.keys()))
        if write_header:
            writer.writeheader()
        writer.writerow(row)

def _load_processed_set() -> set:
    """JSON + 기존 CSV 기반으로 '이미 처리한 URL' 집합 구성(더 안전한 재시작)."""
    _ensure_dirs()
    processed = set()
    # 1) JSON
    if os.path.exists(PROCESSED_JSON):
        try:
            with open(PROCESSED_JSON, "r", encoding="utf-8") as f:
                processed |= set(json.load(f) or [])
        except Exception:
            pass
    # 2) CSV (혹시 JSON 저장 직전에 끊겼던 케이스 보완)
    if os.path.exists(RESULTS_CSV):
        try:
            _df = pd.read_csv(RESULTS_CSV)
            if "상품링크" in _df.columns:
                processed |= set(_df["상품링크"].dropna().astype(str).tolist())
        except Exception:
            pass
    return processed

def _save_processed_set(processed: set):
    _ensure_dirs()
    with open(PROCESSED_JSON, "w", encoding="utf-8") as f:
        json.dump(sorted(list(processed)), f, ensure_ascii=False, indent=2)

# ▷ 접속 제한(서비스 접속이 불가합니다) 감지 & 백오프 재시도
def _is_service_block_page(driver):
    try:
        html = (driver.page_source or "").lower()
    except Exception:
        html = ""
    # 대표 문구 / 버튼 텍스트 키워드
    keys = [
        "현재 서비스 접속이 불가합니다",  # 메인 경고
        "이전 페이지로 가기",
        "새로고침",
        "동시에 접속하는 이용자 수가 많거나",
    ]
    return any(k in html for k in keys)

def wait_if_service_block(driver, base_wait=30, max_wait=600, max_retries=8):
    import time, random
    wait_s = base_wait
    tries = 0
    if not _is_service_block_page(driver):
        return True  # 정상
    while _is_service_block_page(driver) and tries < max_retries:
        jitter = random.uniform(0.7, 1.3)
        sleep_for = min(int(wait_s * jitter), max_wait)
        print(f"⛔ 접속 제한 감지 → {sleep_for}초 대기 후 새로고침 시도 ({tries+1}/{max_retries})")
        time.sleep(sleep_for)
        # 새로고침 버튼 시도 → 실패 시 브라우저 refresh
        clicked = False
        try:
            # 버튼/링크 중 '새로고침' 텍스트를 가진 요소 클릭
            btns = driver.find_elements(By.XPATH, "//*[self::a or self::button][contains(., '새로고침')]")
            if btns:
                btns[0].click()
                clicked = True
        except Exception:
            pass
        if not clicked:
            try:
                driver.refresh()
            except Exception:
                pass
        tries += 1
        wait_s = min(int(wait_s * 1.8), max_wait)
    if _is_service_block_page(driver):
        print("⚠️ 접속 제한 해제 실패: 이 링크는 건너뜁니다.")
        return False
    return True


# ▷ 사람인지 확인(캡차 등) 감지 & 대기
def _is_human_check_page(driver):
    try:
        url = (driver.current_url or "").lower()
    except Exception:
        url = ""
    if any(k in url for k in ["captcha", "nidcaptcha", "/robot"]):
        return True
    try:
        html = (driver.page_source or "").lower()
    except Exception:
        html = ""
    # 대표적인 키워드(국문/영문 혼합)
    keywords = [
        "자동 접속 방지", "자동입력 방지", "자동 접속 차단", "보안문자", "보안 문자를 입력",
        "로봇이 아닙니다", "i am not a robot", "recaptcha", "hcaptcha"
    ]
    return any(k.lower() in html for k in keywords)

def wait_if_human_check(driver, prompt=True, poll_interval=2.0, max_wait_sec=7200):
    # 사람인지 확인 화면이면 사용자 입력을 기다리고, 해결될 때까지 주기적으로 체크
    import time
    if not _is_human_check_page(driver):
        return
    msg = "⚠️ 사람인지 확인 페이지 감지됨. 브라우저에서 인증을 완료하세요."
    if prompt:
        try:
            print(msg, "완료 후 Enter 키를 누르면 즉시 계속합니다.")
            input("인증 완료 후 Enter ▶ ")
            # Enter 이후에도 혹시 남아있으면 폴링으로 재확인
        except Exception:
            print(msg, "(입력 대기 불가 환경) 자동 감지로 전환합니다.")
    # 자동 감지 루프
    start = time.time()
    while _is_human_check_page(driver):
        if time.time() - start > max_wait_sec:
            print("오래 대기하여 다음 링크로 넘어갑니다.")
            break
        time.sleep(poll_interval)


# ▷ 스크롤: 화면 높이의 1.5배를 자연스럽게
def gentle_viewport_scroll(driver, times=1, factor=1.5):
    for _ in range(times):
        vh = driver.execute_script("return window.innerHeight;")
        dist = int(vh * factor)
        # 부드러운 스크롤(이미 smooth_scroll_by가 있다면 그걸 써도 OK)
        try:
            smooth_scroll_by(driver, dist, random.uniform(1.8, 3.2))
        except Exception:
            driver.execute_script(f"window.scrollBy(0,{dist});")
        time.sleep(random.uniform(1.2, 2.2))

# ▷ 표에서 key→value 맵 추출 (th-td 짝으로 읽기)
def table_to_dict(table_elem):
    kv = {}
    rows = table_elem.find_elements(By.CSS_SELECTOR, "tbody > tr")
    for tr in rows:
        ths = tr.find_elements(By.CSS_SELECTOR, "th")
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        # 2열/4열 구조를 모두 커버 (th-td, th-td)
        pairs = []
        # (th,td) 매칭 수 = min(len(ths), len(tds))
        for i in range(min(len(ths), len(tds))):
            key = ths[i].text.strip()
            val = tds[i].text.strip()
            if key:
                kv[key] = val
        # 일부 셀에 colspan이 있어도 위 방식으로 대부분 커버됨
    return kv

# ▷ 상세 페이지 파싱
def parse_detail_page(driver):
    """
    반환: dict
    채워야 할 필드:
      제조사, 브랜드, 모델명, 원산지, 두피타입, 모발타입, 타입,
      제품형태, 용량, 세부제품특징, 향계열, 종류, 성분, 주요제품특징
    """
    data = {
        "제조사":"",
        "브랜드":"",
        "모델명":"",
        "원산지":"",
        "두피타입":"",
        "모발타입":"",
        "타입":"",
        "제품형태":"",
        "용량":"",
        "세부제품특징":"",
        "향계열":"",
        "종류":"",
        "성분":"",
        "주요제품특징":""
    }

    # 1) 기본 스펙표(제조사/브랜드/모델명/원산지 등)
    #    예시 구조 확인: 상단 '상품정보' 표의 th-td (브랜드/원산지/제조사/모델명 등) 
    #    (div.BQJHG3qqZ4 내 table.RCLS1uAn0a 등)  :contentReference[oaicite:3]{index=3} :contentReference[oaicite:4]{index=4}
    base_tables = driver.find_elements(By.CSS_SELECTOR, "div.BQJHG3qqZ4 div.m_PTftTaj7 table")
    for tbl in base_tables:
        kv = table_to_dict(tbl)
        for k, v in kv.items():
            if k in ("제조사","브랜드","모델명","원산지"):
                data[k] = v

    # 2) 세부 속성표(두피타입/모발타입/타입/제품형태/용량/세부제품특징/향계열/종류/성분/주요제품특징)
    #    예시 구조 확인: div.detail_attributes 내 table.RCLS1uAn0a 의 th-td  :contentReference[oaicite:5]{index=5}
    detail_tables = driver.find_elements(By.CSS_SELECTOR, "div.detail_attributes div.m_PTftTaj7 table")
    for tbl in detail_tables:
        kv = table_to_dict(tbl)
        for key in ("두피타입","모발타입","타입","제품형태","용량","세부제품특징","향계열","종류","성분","주요제품특징"):
            if key in kv and not data[key]:
                data[key] = kv[key]

    # 3) 후처리: 괄호 단위 표기 정리 등 필요 시 여기에 추가
    #    예: ml(g) → ml 정규화, 쉼표/공백 트리밍 등
    return data

# ▷ 사람처럼 느리게 이동(다음 링크로 넘어가기 전 대기)
def human_delay(min_s=6.5, max_s=12.0):
    time.sleep(random.uniform(min_s, max_s))

In [6]:
df_list = pd.read_csv('./checkpoints/df_list_checkpoint.csv')
df_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16542 entries, 0 to 16541
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   상품링크    16542 non-null  object 
 1   제목      16542 non-null  object 
 2   원래가격    9048 non-null   float64
 3   현재가격    16542 non-null  int64  
 4   할인율     8499 non-null   float64
 5   스토어     16542 non-null  object 
dtypes: float64(2), int64(1), object(3)
memory usage: 775.5+ KB


In [ ]:
# --- 4. 상세 페이지 스펙 수집 (각 링크 방문 → 1.5화면 스크롤 → 표 파싱) ---

wait = WebDriverWait(driver, 20)

# ===== 본 처리: df_list['상품링크'] 순회 =====

# === [ADD] 재시작을 위한 완료 URL 집합 로드 ===
processed = _load_processed_set()
print("재시작용 완료 목록 로드:", len(processed), "건")

detail_rows = []
links = df_list["상품링크"].fillna("").tolist()

for idx, url in enumerate(links, start=1):
    if not url:
        continue
    if url in processed:
        print(f"[{idx}/{len(links)}] 이미 처리됨 → SKIP: {url}")
        continue

    print(f"[{idx}/{len(links)}] 이동: {url}")
    try:
        driver.get(url)
        # 캡차(사람 확인) 감지 시 잠시 정지 후 재개
        wait_if_human_check(driver)
        # 접속 제한 페이지면 백오프+새로고침으로 해제 시도
        ok = wait_if_service_block(driver)
        if not ok:
            # 건너뛰고 다음 링크로
            human_delay(10.0, 16.0)
            continue
    except Exception as e:
        print("  이동 실패:", e)
        human_delay(6.0, 12.0)
        continue

    # 주요 영역 대기(상품정보 섹션 or 스펙 테이블 등장)
    try:
        # 혹시 이 타이밍에 사람 확인/접속 제한이 떴는지 다시 체크
        ok2 = wait_if_service_block(driver)
        if not ok2:
            human_delay(10.0, 16.0)
            continue
        
        wait_if_human_check(driver)
        wait.until(EC.presence_of_element_located(
            (By.CSS_SELECTOR, "div.BQJHG3qqZ4, div.detail_attributes")
        ))
    except Exception:
        # 그래도 못 찾으면 조금 더 기다렸다가 진행
        time.sleep(3)

    # 첫 화면 안정화 대기 (초기 1–2초 정지)
    time.sleep(random.uniform(1.0, 2.0))

    # 1.5 화면 스크롤(1~2회 정도)
    gentle_viewport_scroll(driver, times=random.randint(1,2), factor=1.5)

    # 상세 페이지 파싱
    parsed = {}
    try:
        parsed = parse_detail_page(driver)
    except Exception as e:
        print("  파싱 오류:", e)
        parsed = {}

    # === [CHANGE] 결과 즉시 체크포인트 저장 (CSV append) ===
    row = {
        "상품링크": url,
        "제조사": parsed.get("제조사",""),
        "브랜드": parsed.get("브랜드",""),
        "모델명": parsed.get("모델명",""),
        "원산지": parsed.get("원산지",""),
        "두피타입": parsed.get("두피타입",""),
        "모발타입": parsed.get("모발타입",""),
        "타입": parsed.get("타입",""),
        "제품형태": parsed.get("제품형태",""),
        "용량": parsed.get("용량",""),
        "세부제품특징": parsed.get("세부제품특징",""),
        "향계열": parsed.get("향계열",""),
        "종류": parsed.get("종류",""),
        "성분": parsed.get("성분",""),
        "주요제품특징": parsed.get("주요제품특징","")
    }
    # 메모리 보관(선택)
    detail_rows.append(row)

    # CSV 즉시 append → 중간 중단에도 누락 최소화
    try:
        _append_row_csv(row, RESULTS_CSV)
    except Exception as _e:
        print("CSV 저장 중 오류(무시하고 진행):", _e)

    # 처리 완료 URL 체크포인트 업데이트
    processed.add(url)
    _save_processed_set(processed)

    # 마지막 체크포인트 로그(디버깅)
    try:
        with open(LAST_LOG, "w", encoding="utf-8") as _f:
            _f.write(f"last_index={idx}\n")
            _f.write(f"processed_count={len(processed)}\n")
            _f.write(f"last_url={url}\n")
    except Exception:
        pass

    # 다음 페이지로 넘어가기 전, 사람 같은 긴 대기 (페이지 차단 완화)
    human_delay(6.0, 12.0)

# === [CHANGE] 이전 실행분까지 포함한 CSV 기준으로 병합 ===
if os.path.exists(RESULTS_CSV):
    df_detail = pd.read_csv(RESULTS_CSV)
    # 혹시 중복이 있으면 최신 행으로 정리
    if "상품링크" in df_detail.columns:
        df_detail = df_detail.drop_duplicates(subset=["상품링크"], keep="last")
else:
    # 첫 실행에서만 메모리 캐시 사용
    df_detail = pd.DataFrame(detail_rows)

print("상세 수집 누적 행수:", len(df_detail))

재시작용 완료 목록 로드: 1611 건
[1/16542] 이미 처리됨 → SKIP: https://ader.naver.com/v1/iJF2KtiXRPKLoHoNfO7avqyG3OhPQz9FIb4ms1DPyyzr6pVWDWROdhJDm6YrZ_6qzvdwacR9S64F3i0eoUEAZKy8c6nKULKM1lFRb9LraJbP_eeCLhHN0h-aKJaEyF_sYNutMxtFT2BiNXM4n2vHusA8swmWRycTlqpz7WeX22ZTi0tJHtVqfS1ZuVxHq2IATJddW-bvl_ZLIXgIK1-03M54-GH3fFKAqGmQ0Y7R4iY0FUyxymSTdT7NVPsbbghlyozfs_jBxEW4Ht2nQRZmJKtxtWecDw1fgE6wp_U5Yqzz0rSLrf6x4fXToJa7QKRS2EVr57b5o17Iv_Jskg6jL5K5fp_x--wZvnJlPP8KKp2HMHw5IAKmVixjqLuTjLBncsm5-Ks-e8D2qrtv6js_TyGANBgUQjGIygUICH1iHSmNDX6Gn7ZtpysDkTHmEs5CntBBhqBdjDHUnmmqbmPSRloHayzTRKKEXuoLupnnTv8=?c=pc.nplusstore.npla&t=0
[2/16542] 이미 처리됨 → SKIP: https://ader.naver.com/v1/KqGKUbXY1m6Dx3FGjvJcSuPHN7lEjQO2rFA8_NTlSCgWQ3BB40C6A0g9Tn7C7zDFRvPNhNJFJs_3qE_MZSQwEaRFdM6CGdqes1SMDl0lmvXdQgqoUGYFl6QCH4Wy9wZfV0VqjjLyQupVXx1WNyfCML9SkG6S8Q3yjFYVKVmtGZzajZABiWSRFD8cnyWaXreQtuI60q9d8KDBa14bo1sAwh1TvosC9meeaAcxd68h8c2Ss9-NOlsRs8y6VYfZ0yYcntk7CZOpl0LxzGUJ4mVwF0Q5-xqeglzUAkKH040p4FgqWRoVaCM6xjLpqGjd95Bnqlu2W56hNy4L0GIaKySlWIlCJ3Mb016fenUjrS

In [ ]:
df_merged = pd.merge(df_list, df_detail, on="상품링크", how="left")
display(df_merged.head(5))

In [ ]:
# ▷ 저장 (원하면 파일명/경로 조정)
out_csv = "naver_hairloss_products_with_specs.csv"
df_merged.to_csv(out_csv, index=False, encoding="utf-8-sig")
print("저장:", out_csv)

In [6]:
# 드라이버 종료
driver.quit()